In [287]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from time import sleep
from random import randint

In [288]:
COLS = ['rank', 'Team', '_current_season', '_last_3', '_last', 'drop1', 'drop2',
               '_last_season', 'Date']

GAMES_ON = [True, False, False, True, False, False, True]

6

In [289]:
def get_date_array():
    arr=[]
    s_parts = STARTDATE.split('-')
    start = datetime.datetime(int(s_parts[0]), int(s_parts[1]), int(s_parts[2]))
    e_parts = ENDDATE.split('-')
    end = datetime.datetime(int(e_parts[0]), int(e_parts[1]), int(e_parts[2]))
    while start <= end:
        if GAMES_ON[start.weekday()]:
            arr.append('%s-%s-%s' % (start.year, start.month, start.day)) 
        start+=datetime.timedelta(days=1)
    return arr
        
get_date_array()

['2021-1-3']

In [290]:
def get_columns(suffix):
    ret = []
    for i in range(len(COLS)):
        if COLS[i][0] == '_':
            ret.append(suffix+COLS[i])
        else:
            ret.append(COLS[i])
    return ret

In [325]:
def get_stats(base_url, suffix):
    dates = get_date_array()
    teams=[]
    columns=get_columns(suffix)
    # Loop through dates
    for d in dates:
        html = urllib.urlopen(base_url + d)
        
        # Sleep to avoid getting blocked
        sleep(randint(1,3))
        soup = BeautifulSoup(html.read())
        html.close()
        stats=soup.find_all('tr')
        
        # form array for single team
        for stat in stats[1:]:
            team = stat.get_text()
            team_row = team.split('\n')[1:-1]
            team_row.append(d)
            
            # append to all data
            teams.append(team_row)
            
    # convert 2D array to df and drop unused data
    df = pd.DataFrame(teams, columns=columns)
    return df.drop(['rank', 'drop1', 'drop2'], axis=1)

In [326]:
# fix teams that don't follow pattern
def fix_extras(team):
    different = {'Los Angeles Rams': 'LA Rams _', 'Los Angeles Chargers': 'LA Chargers _', 
                 'New York Jets': 'NY Jets _', 'New York Giants': 'NY Giants _', 
                 'Washington Football Team': 'Washington _'}
    if team in different.keys():
        return different[team]
    return team

# fix teams that do follow pattern
def get_cities(team):
    return (' ').join(team.split(' ')[:-1])

# fix dates
def format_dates(date):
    t = date.split('/')
    return '%s-%s-%s' % (t[2], t[0], t[1])

In [327]:
# get games csv
def get_games(path):
    GAMES = pd.read_csv(path)
    GAMES['team_home'] = GAMES['team_home'].map(fix_extras)
    GAMES['team_away'] = GAMES['team_away'].map(fix_extras)
    GAMES['team_home'] = GAMES['team_home'].map(get_cities)
    GAMES['team_away'] = GAMES['team_away'].map(get_cities)
    GAMES['schedule_date'] = GAMES['schedule_date'].map(format_dates)
    return GAMES

In [328]:
# get nets from 2d array of dfs to cross net score and names
def get_nets(cross, names_c):
    ret=GAMES
    for i in range(len(cross)):
        ret = pd.merge(ret, cross[i][0], left_on=['schedule_date', 'team_home'], right_on=['Date', 'Team'], how='left')
        ret = pd.merge(ret,  cross[i][1], left_on=['schedule_date', 'team_away'], right_on=['Date', 'Team'], how='left')
        ret = ret.drop(['Team_x' , 'Team_y', 'Date_x', 'Date_y'], axis=1)

        suf1 = names_c[i][0]
        suf2 = names_c[i][1]

        col1 = get_columns(suf1)
        col2 = get_columns(suf2)

        col1 = [i for i in col1 if i[:len(suf1)]==suf1]
        col2 = [i for i in col2 if i[:len(suf2)]==suf2]



        for c in range(len(col1)):
            neutral_suf = 'N' + suf1[:-1] + col1[c][3:]

            ret[neutral_suf] = ret[col1[c]].astype(float) - ret[col2[c]].astype(float)
            ret=ret.drop([col1[c], col2[c]], axis=1)

    return ret

### INFO FOR DATA TO GET

In [ ]:
STARTDATE = '2021-01-03'
ENDDATE = '2021-01-03' # inlusive
CSV_TITLE = 'differences_data_2021_01_06.csv'

In [329]:
GAMES = get_games('data/delete.csv')

PYD = get_stats('https://www.teamrankings.com/nfl/stat/opponent-passing-yards-per-game?date=', 'PYD')
PYO = get_stats('https://www.teamrankings.com/nfl/stat/passing-yards-per-game?date=', 'PYO')
RYO = get_stats('https://www.teamrankings.com/nfl/stat/rushing-yards-per-game?date=', 'RYO')
RYD = get_stats('https://www.teamrankings.com/nfl/stat/opponent-rushing-yards-per-game?date=', 'RYD')

In [330]:
cross = [[PYO, PYD], [RYO, RYD]]
names_c = [['PYO', 'PYD'], ['RYO', 'RYD']]

data=get_nets(cross, names_c)

In [331]:
data

,schedule_date,team_home,team_away,net_score,total_points,spread_favorite,over_under_line,NPY_current_season,NPY_last_3,NPY_last,NPY_last_season,NRY_current_season,NRY_last_3,NRY_last,NRY_last_season
0,2021-1-3,Buffalo,Miami,30.0,82.0,-3.0,42.5,39.8,24.0,30.0,-57.6,-9.2,33.7,26.0,-4.5
1,2021-1-3,Carolina,New Orleans,-26.0,40.0,-6.0,46.5,27.7,-4.0,-107.0,-12.7,13.5,-54.0,23.0,19.8
2,2021-1-3,Chicago,Green Bay,-19.0,51.0,-4.5,48.5,7.7,35.7,159.0,-19.6,-10.5,58.0,-28.0,-37.6
3,2021-1-3,Cincinnati,Baltimore,-35.0,41.0,-13.5,45.0,-5.8,-39.3,156.0,28.4,-4.8,56.0,115.0,-5.8
4,2021-1-3,Cleveland,Pittsburgh,2.0,46.0,-10.5,44.5,28.6,116.3,16.0,27.5,39.4,-31.4,-82.0,9.2
5,2021-1-3,Denver,Las Vegas,-1.0,63.0,-2.5,50.5,-50.7,-52.0,10.0,-62.0,-6.4,-23.0,3.0,5.8
6,2021-1-3,Detroit,Minnesota,-2.0,72.0,-3.0,53.5,-2.5,-17.0,-206.0,16.8,-43.5,-107.7,-191.0,-8.6
7,2021-1-3,Houston,Tennessee,-3.0,79.0,-7.0,55.5,6.6,11.7,108.0,-17.6,-31.2,-30.3,-68.0,15.9
8,2021-1-3,Indianapolis,Jacksonville,14.0,42.0,-15.5,48.0,-11.8,-4.3,-25.0,-41.8,-30.4,-23.4,-1.0,-6.2
9,2021-1-3,Kansas City,LA Chargers,-17.0,59.0,-6.5,43.0,85.0,25.3,15.0,82.5,-7.9,4.7,-16.0,-11.3
